In [ ]:
!pip uninstall -y enum34
!pip install --upgrade pip
!pip install farm

In [1]:
!set NotebookApp.iopub_data_rate_limit=2000000.0

In [2]:
# fmt: off
import logging
from pathlib import Path

from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import TextClassificationProcessor
from farm.modeling.optimization import initialize_optimizer
from farm.infer import Inferencer
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import MultiLabelTextClassificationHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.utils import set_all_seeds, MLFlowLogger, initialize_device_settings
import os.path
import numpy as np
import tensorflow as tf
import farm
import torch

06/22/2021 18:25:36 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
06/22/2021 18:25:55 - WARNING - tensorflow -   From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



In [3]:
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO)

ml_logger = MLFlowLogger(tracking_uri="./FARM_logs/final_prediction/")
ml_logger.init_experiment(experiment_name='classification', run_name="run_text_classification")

set_all_seeds(seed=123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

device, n_gpu = initialize_device_settings(use_cuda=torch.cuda.is_available())
n_epochs = 6
batch_size = 526

evaluate_every = 4000
lang_model = "distilbert-base-german-cased"
do_lower_case = True

tokenizer = AutoTokenizer.from_pretrained(lang_model)
model = AutoModel.from_pretrained(lang_model)


 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 
INFO: 'classification' does not exist. Creating a new experiment


06/22/2021 18:26:00 - INFO - farm.utils -   Using device: CUDA 
06/22/2021 18:26:00 - INFO - farm.utils -   Number of GPUs: 8
06/22/2021 18:26:00 - INFO - farm.utils -   Distributed Training: False
06/22/2021 18:26:00 - INFO - farm.utils -   Automatic Mixed Precision: None


Devices available: cuda


In [4]:
tokenizer = Tokenizer.load(
        pretrained_model_name_or_path=lang_model,
        do_lower_case=do_lower_case)

06/22/2021 18:26:06 - INFO - filelock -   Lock 140552330730352 acquired on /home/ec2-user/.cache/huggingface/transformers/8d669fffc251fa3f490617e99cbe9777175fb84575b9e474aa9b5c04dd37a68d.381bf7fa8fc6b6a863694b3044643c123febc918ddb39b34a48285796ecf04bf.lock


Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

06/22/2021 18:26:06 - INFO - filelock -   Lock 140552330730352 released on /home/ec2-user/.cache/huggingface/transformers/8d669fffc251fa3f490617e99cbe9777175fb84575b9e474aa9b5c04dd37a68d.381bf7fa8fc6b6a863694b3044643c123febc918ddb39b34a48285796ecf04bf.lock
06/22/2021 18:26:06 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'DistilBertTokenizer'
06/22/2021 18:26:07 - INFO - filelock -   Lock 140552330267448 acquired on /home/ec2-user/.cache/huggingface/transformers/5bb297aad6870764f744be4eeecd4679617a15931cc1e0b373bd23919defdeea.4d65bbd3b91f2762e9d2c779d48ab14052439d3fcc8c3d2fe78c7322a9ac8d64.lock


Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

06/22/2021 18:26:07 - INFO - filelock -   Lock 140552330267448 released on /home/ec2-user/.cache/huggingface/transformers/5bb297aad6870764f744be4eeecd4679617a15931cc1e0b373bd23919defdeea.4d65bbd3b91f2762e9d2c779d48ab14052439d3fcc8c3d2fe78c7322a9ac8d64.lock
06/22/2021 18:26:07 - INFO - filelock -   Lock 140561127680152 acquired on /home/ec2-user/.cache/huggingface/transformers/a63f1d64fc50ca021f544776df546258fb07cf1b516893a586ddabbbfc0cf924.427a4a69da815751f698c649e071f9d0c6c9f0e6630a2da72e22342b013c8d3d.lock


Downloading:   0%|          | 0.00/479k [00:00<?, ?B/s]

06/22/2021 18:26:08 - INFO - filelock -   Lock 140561127680152 released on /home/ec2-user/.cache/huggingface/transformers/a63f1d64fc50ca021f544776df546258fb07cf1b516893a586ddabbbfc0cf924.427a4a69da815751f698c649e071f9d0c6c9f0e6630a2da72e22342b013c8d3d.lock
06/22/2021 18:26:09 - INFO - filelock -   Lock 140561127340688 acquired on /home/ec2-user/.cache/huggingface/transformers/dac255dfe6b8fb9ca00b63c716bca62de0c5044d016751583ea34f2d39606023.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

06/22/2021 18:26:09 - INFO - filelock -   Lock 140561127340688 released on /home/ec2-user/.cache/huggingface/transformers/dac255dfe6b8fb9ca00b63c716bca62de0c5044d016751583ea34f2d39606023.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock


In [5]:
import pandas as pd
label_list = pd.read_csv('labels_new_distilbert.csv')
label_list = label_list['0'].values.tolist()

In [ ]:
label_list
print(label_list)


In [7]:
metric = "acc"

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=200,
                                            data_dir=Path("./DataSetCreation/"),
                                            label_list=label_list,
                                            label_column_name="label",
                                            metric=metric,
                                            quote_char='"',
                                            multilabel=True,
                                            train_filename="train_newtext.tsv",
                                            dev_filename="val_newtext.tsv",
                                            test_filename="test_newtext.tsv",
                                            dev_split=0,
                                            )

06/22/2021 18:26:12 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids


In [ ]:
# 3. Create a DataSilo that loads several datasets (train/dev/test), provides DataLoaders for them and calculates a few descriptive statistics of our datasets
data_silo = DataSilo(
        processor=processor,
        batch_size=batch_size)

In [ ]:
## 4. Create an AdaptiveModel
# a) which consists of a pretrained language model as a basis
language_model = LanguageModel.load(lang_model)
# b) and a prediction head on top that is suited for our task => Text classification
prediction_head = MultiLabelTextClassificationHead(class_weights=data_silo.calculate_class_weights(task_name='text_classification'),
    num_labels=len(label_list))

model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=0.1,
        lm_output_types=["per_sequence"],
        device=device)

# 5. Create an optimizer
model, optimizer, lr_schedule = initialize_optimizer(
        model=model,
        learning_rate= 3e-5,
        device=device,
        n_batches=len(data_silo.loaders["train"]),
        n_epochs=n_epochs)
 # 6. Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
trainer_6_epochs_newtext_imbalanced = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device)

In [ ]:
trainer_6_epochs_newtext_imbalanced.train()

06/22/2021 18:28:29 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 552/552 [09:13<00:00,  1.00s/it]
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples w

06/23/2021 03:06:07 - INFO - farm.eval -   task_name: text_classification
06/23/2021 03:06:07 - INFO - farm.eval -   acc: 0.9507767415208161
06/23/2021 03:06:07 - INFO - farm.eval -   report: 
                   precision    recall  f1-score   support

         Germany     0.9852    0.9197    0.9513    105591
            Asia     0.9369    0.8680    0.9011     16464
    North_Europe     0.8292    0.9079    0.8668       369
  Eastern_Europe     0.8984    0.8639    0.8808     10370
  Western_Europe     0.8206    0.8566    0.8382      3327
    South_Europe     0.8834    0.8644    0.8738      4225
   North_America     0.7294    0.8763    0.7961       889
   South_America     0.8298    0.9033    0.8650      1355
          Africa     0.8951    0.8571    0.8757      5059
         Oceania     0.8349    0.8667    0.8505       105
         married     0.9086    0.7056    0.7943     37649
     partnership     0.8194    0.6068    0.6972     20984
       unmarried     0.9531    0.7470    0.8375    

WrappedDataParallel(
  (module): AdaptiveModel(
    (language_model): DistilBert(
      (model): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(31102, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0): TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm): LayerNorm((768,), eps=1e

In [11]:
# 8. Hooray! You have a model. Store it:
save_dir = Path("./saved_models/distilbert/6epochs_final")
model.save(save_dir)
processor.save(save_dir)